## Rapport pour agréger les résultats des méthodes TABU

note : Nous avons tenté d'intégrer l'aspiration à nos méthodes (ignorer la liste tabu si l'opération donne un voisin plus grand que la meilleure solution jamais trouvée, mais ce n'était pas probant, et plus long, donc on a rien sauvegardé et tout laissé ailleurs)

### Différences Algorithmiques

- Tabu 1 est en gros le tabu du cours, sauf qu'il a un paramètre pour lui laisser dépasser la capacité du sac à dos dans ses recherches. Il peut accepter des valeurs 10% plus grandes que la taille du sac, etc.
- tabu 2 compte le poids au dessus de la capacité du sac a dos, le multiplie par un facteur et le soustrait à la valeur de la somme des fitnesss des items du sac. Le facteur est paramétrable

De manière générale, il semble que tabu 2 soit plus efficace pour trouver des solutions optimale, surtout sur les pi-13...

#### LES TABLEAUX SONT NOS AMIS

Les 3 classes TABU_NUMPY, TABU_openCL et tabu_opencl2 donnent les mêmes résultats avec les mêmes données de départ.

Ils sont tous basé sur TABU2 (sans héritage car le ctrl-c ctrl-v existe), et sont sensés donner des résultats quasi-identiques à celui ci (cf [ce fichier](tabu_comp_np_ocl_class.py))

Ils fonctionnent en calculant avect quelques opérations sur des tableaux les différences de fitness des N voisins possibles.

Ensuite, on récupère l'indice de la meilleure valeur pour avoir le meilleur voisin (en prenant en compte la liste tabu )

- tabu numpy le fait avec numpy
- tabu opencl calcul les voisins sur la carte graphique avec pyopencl, puis calcul le meilleur avec numpy
- tabu opencl2 fait tout sur la carte graphique avec des supers algos de réductions

Au final, numpy est le plus efficace : environ 80 fois plus rapide que le tabu 2 classique.

OpenCL2 quand à lui est environ 2 fois plus long que numpy (et encore ce n'est en prenant en compte que la résolution d'un problème, pas la compilation du programme etc...).

Dans un pc avec un CPU faible ou très occupé, opencl pourrait être plus rapide. Il pourrait également l'être si on utilise la classe Testor et qu'on teste autant de solutions  en parallèle que le CPU n'a de coeurs : alors numpy ne pourrait pas aussi bien profiter du multi-processing, et utiliser la carte graphique pourrait être plus intéressant. Enfin pour l'instant avec toutes les compilations qui se feraient à la chaine, pas sur... (Je tente de trouver des justifications à ces heures de développement dans le cadre de ce projet 👀)


### Méthodo

Déjàs on a mis en place un système de seed (après un certain temps) pour que les tests soient reproductibles (toujours sur 1 pour les tabou, la seed par défaut).... Il n'a pas été utilisé sur tabu 1, mais l'a été sur une bonne part de tabu 2.

On a fait varier les paramètres 1 par un, avec peu d'itérations de tests pour avoir des idées globales.

Ensuite en repartant des paramètes qui marchent à peu près bien pour tous (plus trop de traces de ces recherches), on a fait varier :
1. taille de la liste TABU (avec trop d'itérations à priori)
2. nombre d'itération
3. facteur overflow  ou poids max
4. taille initiale
5. le nombre d'itérations à nouveau, ou d'autres paramètres en fonction.

Vous pouvez aller voir les fichiers TABUX_100XX.ipynb pour voir la majorités des tests effectués pour trouver les paramètres optimaux, et des jolis graphiques avec l'évolution de la fitness en fonction de la valeur du paramètre qui varie.

### Classe Testor

La classe testor permet de tester N changements d'un paramètre d'un `Solver`. Ces solvers sont nos algorithmes TABU, génétiques etc.

On lui passe un solver préparer, et une fonction pour le réinitialiser, ansi qu'un itérateur qui permet de faire évoluter le paramètre et un entier, qui défini combien de tests doivent être effectués pour chaque valeur du paramètre. La seed donnée au départ doit être augmentée de 1 par la fonction qui donne le nouveau Solver, pour que le cas de test ne soit pas toujours identique (mais reproductible)

Les différentes valeurs de paramètres sont testées sur des threads indépendants.

Pour chaque groupe de test (chaque batch), sont calculés : la médianne, le 20ème percentile, le 80ème percentile ainsi que la moyenne.
Ainsi on a des infos plutôt complètes sur ce qui s'y passe, sans saturer le graphique.

### rappel sur la Lecture des graphiques

Au fil des fichiers TABUX_100XX.ipynb, des graphiques sont affichés. ceux ci prennent directement en paramètre ce qui est renvoyé de Testor.

On y voit la moyenne avec un + rouge, la médianne, le point bleu, et les percentile, les 2 traits verticaux autour de la médianne.


### Paramètres Optimaux

#### pour 100 item :
les chiffres après représentent le sac à dos associé au paramètre (1 pour le 12, 2 pour 13, 3 -> 15)

In [ ]:
#tabu 1
param_tabu1_12_100 = {"iter_max": 200,"tabu_size": 60, "max_weight": 1, "def_sol_size":  0.01}
param_tabu1_13_100 = {"iter_max": 2200,"tabu_size": 95,"max_weight": 1,"def_sol_size":  0.01}#pas sur de la reproductibilité sur d'autres sac de la même forme
param_tabu1_15_100 = {"iter_max": 200,"tabu_size": 60,"max_weight": 1,"def_sol_size":  0.01}
#tabu 2
param_tabu2_12_100 = {"iter_max": 200,"tabu_size": 60,"cout_depassement": 2.5,"def_sol_size": 0.001}
param_tabu2_13_100 = {"iter_max": 200,"tabu_size": 60,"cout_depassement": 2.5,"def_sol_size": 0.001}
param_tabu2_15_100 = {"iter_max": 200,"tabu_size": 60,"cout_depassement": 2.5,"def_sol_size": 0.001}

#### Paramètres pour 1000 items

In [ ]:
#tabu 1
param_tabu1_12_1000 = {"iter_max": 5000, "tabu_size": 420, "max_weight": 1.0, "def_sol_size": 0.001}
param_tabu1_13_1000 = {"iter_max": 3500, "tabu_size": 960, "max_weight": 1.0, "def_sol_size": 0.55}
param_tabu1_15_1000 = {"iter_max": 3500, "tabu_size": 100, "max_weight": 1.0, "def_sol_size": 0.15}
#tabu 2
param_tabu2_12_1000 = {"iter_max": 500, "tabu_size": 200, "cout_depassement": 2.75, "def_sol_size": 0.01}
param_tabu2_13_1000 = {"iter_max": 550, "tabu_size": 200, "cout_depassement": 2.75, "def_sol_size": 0.01}
param_tabu2_15_1000 = {"iter_max": 400, "tabu_size": 105, "cout_depassement": 1.24, "def_sol_size": 1.4}

#### pour 10k items

In [ ]:
# Tabu 1
param_tabu1_12_10000 = {"iter_max": 3000, "tabu_size": 420, "max_weight": 1.0, "def_sol_size": 0.001}
param_tabu1_13_10000 = {"iter_max": 3000, "tabu_size": 960, "max_weight": 1.0, "def_sol_size": 1.0}
param_tabu1_15_10000 = {"iter_max": 3000, "tabu_size": 100, "max_weight": 1.0, "def_sol_size": 1.0}
#tabu 2
param_tabu2_12_10000 = {"iter_max": 2500, "tabu_size": 800, "cout_depassement": 1.1, "def_sol_size": 0.3}
param_tabu2_13_10000 = {"iter_max": 9000, "tabu_size": 1800, "cout_depassement": 2.35, "def_sol_size": 0.3}
#param_tabu2_13_10000 = {"iter_max": 1000, "tabu_size": 800, "cout_depassement": 1.8, "def_sol_size": 0.3} # -> ça marche bien mais c'est pas parfait (seulement 99%)
param_tabu2_15_10000 = {"iter_max": 9000, "tabu_size": 1000, "cout_depassement": 1.1, "def_sol_size": 0.3}

### Qualité des résultats

à noter : nous avons chercher les résultats optimaux pour chaque cas précis, et pas de manière générale, ni en fonction de la composition de chaque sacs...

Alors qu'on aurait pu imaginer des équations pour trouver le nombre d'itérations maximum ou le facteur du cout de dépassement (par exemple, ratio moyen des 20% des meilleurs item + 10%, qui serait fonctionnel sur les 3 sacs à 10k par exemple)

On compare les résultats à ceux donnés par le `knapsack_solver` de `Ortools`, implémentés avec cette façade : [Other_solver](tools/Other_solver.py)
Ces résultats sont sensés être parfaits

la qualité a été testée dans [cet ipynb](TABU_test_quality.ipynb)

- par sac

| %         | Tabu 1 | Tabu 2 | Temps 1 (s) | Temps 2 (s) | Ratio 2/1 |
|-----------|--------|--------|------------------|------------------|-----------|
| 12-100    | 100    | 100    | 1.403            | 2.644            | 0.53      |
| 12-1000   | 100    | 100    | 27.512           | 0.178            | 154.93    |
| 12-10000  | 100    | 100    | 76.037           | 0.934            | 81.38     |
| 13-100    |   98.04  | 100   |  0.376            |   2.821           | 0.13     |
| 13-1000   | 67.51  | 100    | 7.044            | 0.263            | 26.81     |
| 13-10000  | 64.39  | 99.94  | 73.119           | 4.933            | 14.82     |
| 15-100    | 99.1 | 99.41   |   1.156         |  2.725          | 0.42      |
| 15-1000   | 97.55  | 99.21  | 26.783           | 0.142            | 189.07    |
| 15-10000  | 97.54  | 99.19  | 76.641           | 3.517            | 21.79     |

- par nb_item

| %         | Tabu 1 | Tabu 2 | Temps 1 (s) | Temps 2 (s) | Ratio 2/1 |
|-----------|--------|--------|------------------|------------------|-----------|
| 12-100    | 100    | 100    | 1.403            | 2.644            | 0.53      |
| 13-100    |   98.04  | 100   |  0.376            |   2.821           | 0.13     |
| 15-100    | 99.1 | 99.41   |   1.156         |  2.725          | 0.42      |
| 12-1000   | 100    | 100    | 27.512           | 0.178            | 154.93    |
| 13-1000   | 67.51  | 100    | 7.044            | 0.263            | 26.81     |
| 15-1000   | 97.55  | 99.21  | 26.783           | 0.142            | 189.07    |
| 12-10000  | 100    | 100    | 76.037           | 0.934            | 81.38     |
| 13-10000  | 64.39  | 99.94  | 73.119           | 4.933            | 14.82     |
| 15-10000  | 97.54  | 99.19  | 76.641           | 3.517            | 21.79     |

à noter : les temps sont à diviser par 200 pour les 100 items et 4 pour les 1000 items

à renoter : les pourcentages donnés sont la moyenne des pourcentages des résultats, pas la médiane ou un percentile -> par exemple on peut explorer 10 fois 15-100 en paralèle avec TABU 2, et on a toutes les chances de tomber sur un résultat optimal (plus de 80% de chance à chaque tirage), ça prendrait 0.14 secondes environ.
#### constat

Il semble que la méthode Tabu 2 soit globalement meilleure, à la fois algorithmiquement, puisqu'elle trouve de meilleurs résultats (surtout pour les tailles autour de 1000, elle le fait avec beaucoup moins d'itérations), et beaucoup plus vite dès que le nombre d'item augmente, même si c'est surtout grace à numpy, qu'on aurait aussi pu implémenter sur tabu classique.

D'ailleurs plus la taille est grande moins bonnne sont nos solutions... et c'est "logique" : mais on trouve quand même régulièrement les bonnes solution sur 12-1000, cf [ce fichier de test](TABU2_10000_i.ipynb) : on y voit qu'à partir de 8500 itérations, on trouve le résultat optimal dans grand minimum 80% des cas (car les barres bleues affichées sont les percentiles 20 et 80, et qu'elles se confondent avec la solution optimale)